In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_pos.csv').drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_pos_HCD35+decomp.sav', 'wb'))

In [5]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_pos_HCD35+decomp.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29038
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.2103
[3]	valid_0's multi_logloss: 1.14555
[4]	valid_0's multi_logloss: 1.08958
[5]	valid_0's multi_logloss: 1.04139
[6]	valid_0's multi_logloss: 0.998608
[7]	valid_0's multi_logloss: 0.959936
[8]	valid_0's multi_logloss: 0.92686
[9]	valid_0's multi_logloss: 0.894707
[10]	valid_0's multi_logloss: 0.863355
[11]	valid_0's multi_logloss: 0.833659
[12]	valid_0's multi_logloss: 0.810613
[13]	valid_0's multi_logloss: 0.78459
[14]	valid_0's multi_logloss: 0.767057
[15]	valid_0's multi_logloss: 0.742304
[16]	valid_0's multi_logloss: 0.723718
[17]	valid_0's multi_logloss: 0.707868
[18]	valid_0's multi_logloss: 0.690606
[19]	valid_0's multi_logloss: 0.673918
[20]	valid_0's multi_logloss: 0.658921
[21]	valid_0's multi_logloss: 0.645875
[22]	valid_0's multi_logloss: 0.632623
[23]	valid_0's multi_logloss: 0.621649
[24]	valid_0's multi_logloss: 0.611278
[25]	valid_0's multi

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [7]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
pickle.dump(model, open('../model/xgb_pos_HCD35+decomp.sav', 'wb'))
t.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_HCD35+decomp.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.9195 - acc: 0.2261 - val_loss: 1.4578 - val_acc: 0.2173
Epoch 2/100
1340/1340 [==============================] - 1s 803us/step - loss: 1.7867 - acc: 0.2746 - val_loss: 1.4025 - val_acc: 0.2619
Epoch 3/100
1340/1340 [==============================] - 1s 761us/step - loss: 1.7028 - acc: 0.2649 - val_loss: 1.3275 - val_acc: 0.3185
Epoch 4/100
1340/1340 [==============================] - 1s 743us/step - loss: 1.6304 - acc: 0.2963 - val_loss: 1.2811 - val_acc: 0.3393
Epoch 5/100
1340/1340 [==============================] - 1s 771us/step - loss: 1.6103 - acc: 0.3127 - val_loss: 1.2672 - val_acc: 0.4286
Epoch 6/100
1340/1340 [==============================] - 1s 744us/step - loss: 1.5778 - acc: 0.3239 - val_loss: 1.2709 - val_acc: 0.4821
Epoch 7/100
1340/1340 [==============================] - 1s 796us/step - loss: 1.5257 - acc: 0.3179 - val_loss: 1.2825 - val_acc: 0.4

1340/1340 [==============================] - 1s 731us/step - loss: 0.2251 - acc: 0.9201 - val_loss: 1.1519 - val_acc: 0.6399
Epoch 61/100
1340/1340 [==============================] - 1s 736us/step - loss: 0.2368 - acc: 0.9179 - val_loss: 1.1630 - val_acc: 0.6399
Epoch 62/100
1340/1340 [==============================] - 1s 781us/step - loss: 0.2184 - acc: 0.9239 - val_loss: 1.0798 - val_acc: 0.6696
Epoch 63/100
1340/1340 [==============================] - 1s 777us/step - loss: 0.2060 - acc: 0.9299 - val_loss: 1.0566 - val_acc: 0.6756
Epoch 64/100
1340/1340 [==============================] - 1s 765us/step - loss: 0.1805 - acc: 0.9239 - val_loss: 1.1212 - val_acc: 0.6548
Epoch 65/100
1340/1340 [==============================] - 1s 715us/step - loss: 0.2257 - acc: 0.9254 - val_loss: 1.1863 - val_acc: 0.6369
Epoch 66/100
1340/1340 [==============================] - 1s 703us/step - loss: 0.2336 - acc: 0.9172 - val_loss: 1.1278 - val_acc: 0.6488


In [9]:
f = []
f.append(t)

In [10]:
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_neg.csv').drop('Subclass', axis=1)
], axis=1)

In [11]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [12]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_ng_HCD35+decomp.sav', 'wb'))

In [13]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_ng_HCD35+decomp.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29219
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21019
[3]	valid_0's multi_logloss: 1.14622
[4]	valid_0's multi_logloss: 1.08733
[5]	valid_0's multi_logloss: 1.03251
[6]	valid_0's multi_logloss: 0.992919
[7]	valid_0's multi_logloss: 0.95358
[8]	valid_0's multi_logloss: 0.920615
[9]	valid_0's multi_logloss: 0.886328
[10]	valid_0's multi_logloss: 0.861479
[11]	valid_0's multi_logloss: 0.836112
[12]	valid_0's multi_logloss: 0.81338
[13]	valid_0's multi_logloss: 0.792016
[14]	valid_0's multi_logloss: 0.768451
[15]	valid_0's multi_logloss: 0.74962
[16]	valid_0's multi_logloss: 0.732339
[17]	valid_0's multi_logloss: 0.718489
[18]	valid_0's multi_logloss: 0.707162
[19]	valid_0's multi_logloss: 0.697014
[20]	valid_0's multi_logloss: 0.688042
[21]	valid_0's multi_logloss: 0.677015
[22]	valid_0's multi_logloss: 0.664416
[23]	valid_0's multi_logloss: 0.655276
[24]	valid_0's multi_logloss: 0.648528
[25]	valid_0's multi

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [15]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
t.append(round(model.score(X_test, y_test)*100, 2))
pickle.dump(model, open('../model/xgb_ng_HCD35+decomp.sav', 'wb'))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_HCD35+decomp.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 4ms/step - loss: 2.2399 - acc: 0.2011 - val_loss: 1.4254 - val_acc: 0.3617
Epoch 2/100
373/373 [==============================] - 0s 250us/step - loss: 1.9099 - acc: 0.2547 - val_loss: 1.3959 - val_acc: 0.4149
Epoch 3/100
373/373 [==============================] - 0s 253us/step - loss: 1.6998 - acc: 0.2842 - val_loss: 1.4462 - val_acc: 0.4255
Epoch 4/100
373/373 [==============================] - 0s 282us/step - loss: 1.7601 - acc: 0.3298 - val_loss: 1.5122 - val_acc: 0.4574
Epoch 5/100
373/373 [==============================] - 0s 337us/step - loss: 1.6800 - acc: 0.3566 - val_loss: 1.5430 - val_acc: 0.4681
Epoch 6/100
373/373 [==============================] - 0s 362us/step - loss: 1.6782 - acc: 0.3780 - val_loss: 1.5367 - val_acc: 0.4681
Epoch 7/100
373/373 [==============================] - 0s 312us/step - loss: 1.6358 - acc: 0.3378 - val_loss: 1.4952 - val_acc: 0.4681
Epoch 8/100


In [17]:
f.append(t)

In [18]:
# save to csv
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index= ['Decomposition + HCD35_Positive', 'Decomposition + HCD35_Negative']
)

,Random Forest,LightGBM,XGBoost,Keras
Decomposition + HCD35_Positive,69.94,83.63,80.65,64.88
Decomposition + HCD35_Negative,69.15,72.34,78.72,62.77


In [19]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index= ['Decomposition + HCD35_Positive', 'Decomposition + HCD35_Negative']
).T.to_csv('../result/Decomposition + HCD35.csv')